In [1]:
import time
import rand_scraper

In [2]:
from rand_scraper import get_report_urls
report_urls = get_report_urls(begin_page=1, end_page=1, verbose=True)
print('%d report_urls for all report' % len(report_urls))

24 report_urls for all report


In [3]:
from rand_scraper import get_blog_urls
blog_urls = get_blog_urls(verbose=True)
print('%d report_urls for all report' % len(blog_urls))

443 report_urls for all report


In [4]:

def parse_report(url):
    def parse_author(soup):
        author = soup.find('p', class_='authors')
        if not author:
            return 'no author'
        return author.text

    def parse_title(soup):
        title = soup.find('h1', id='RANDTitleHeadingId').text
        if not title:
            return ''
        return title

    def parse_date(soup):
        date = soup.find('meta',attrs={'name':"rand-date"})['content']
        if not date:
            return '20190306'
        return date

    def parse_content(soup):
        content = soup.find('div', class_= 'product-main').text
        if not content:
            return ''
        return content

    def parse_publication_link(soup):
        for a in soup.select('a'):
            if '/content/dam/rand/' in a.attrs.get('href', ''):
                return a.attrs['href']
            if 'external' in a.attrs.get('class', ''):
                return 'external article'

    soup = get_soup(url)
    temp_content_url = parse_publication_link(soup)
    if '/content/dam/rand/' in temp_content_url:
        if 'https:' not in temp_content_url:
            content_url = 'https://www.rand.org' + temp_content_url
    else:
        content_url = temp_content_url
    return {
        'url': url,
        'title': parse_title(soup),
        'date': parse_date(soup),
        'author': parse_author(soup),
        'abstract': parse_content(soup),
        'content_url': content_url
    }

In [5]:
from rand_scraper import get_soup
get_soup('https://www.pacforum.org/programs/pacific-forum/publications/pacnet-newsletter').find('article', role= 'article').find('h1').text

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
from rand_scraper import parse_page
def pprint(json_object):
    for k, v in json_object.items():
        print('{} : {} ..'.format(k, str(v)[:100]))
    print('\n')

SLEEP = 0.5

for url in report_urls[:40]:
    rjson_object = parse_page(url)
    pprint(rjson_object)

for url in blog_urls[:1]:
    rjson_object = parse_page(url)
    pprint(rjson_object)

In [ ]:
from rand_scraper import yield_latest_report

begin_date = '2018-12-01'
max_num = 10
sleep = 1.0

for i, json_obj in enumerate(yield_latest_report(begin_date, max_num, sleep)):
    title = json_obj['title']
    time = json_obj['date']
    print('[{} / {}] ({}) {}'.format(i+1, max_num, time, title))

In [ ]:
from rand_scraper import yield_latest_blog

begin_date = '2018-12-01'
max_num = 10
sleep = 1.0

for i, json_obj in enumerate(yield_latest_blog(begin_date, max_num, sleep)):
    title = json_obj['title']
    time = json_obj['date']
    print('[{} / {}] ({}) {}'.format(i+1, max_num, time, title))